In [9]:
pip install okx

Note: you may need to restart the kernel to use updated packages.


# 要購買的價格

In [150]:
from okx.app import OkxSWAP
from okx.app import MarketSWAP
from okx.app import Market as Market_okx
from okx.api import Account
from okx.api import Trade
from okx_api import Market as Market_api
from pprint import pprint
import json
import os
import time
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
KEY = os.getenv('key')
SECRET = os.getenv('secret')
PASSPHRASE = os.getenv('passphrase')

okxSWAP = OkxSWAP(
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
)

[SUCCESS] 设置持仓方式为双向持仓成功，posMode="long_short_mode"


In [89]:
market = Market_api(key = '', secret = '', passphrase = '', flag = '0')
# 获取所有产品行情信息
result = market.get_history_index_candles(instId = "INJ-USDT", bar = "1m", limit = "10")
pprint(result['data'])
    
val = (float(result['data'][0][1]) - float(result['data'][9][1]))/float(result['data'][9][1])*100 #最新資料減第十個資料漲跌百分比
# 如果是跌幅
# float(result['data'][9][1])*(1 - abs(val)/100)
# 如果是漲幅
# float(result['data'][2][1])*(1 + abs(val)/100)

[['1705207860000', '37.752', '37.752', '37.628', '37.632', '1'],
 ['1705207800000', '37.79', '37.809', '37.751', '37.753', '1'],
 ['1705207740000', '37.771', '37.798', '37.766', '37.793', '1'],
 ['1705207680000', '37.751', '37.781', '37.751', '37.771', '1'],
 ['1705207620000', '37.813', '37.824', '37.743', '37.749', '1'],
 ['1705207560000', '37.811', '37.844', '37.8', '37.812', '1'],
 ['1705207500000', '37.872', '37.873', '37.806', '37.81', '1'],
 ['1705207440000', '37.836', '37.873', '37.834', '37.87', '1'],
 ['1705207380000', '37.86', '37.864', '37.823', '37.834', '1'],
 ['1705207320000', '37.825', '37.9', '37.818', '37.854', '1']]


In [79]:
account = Account(
    # 交易账户需要秘钥
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
    # 实盘：0，虚拟盘：1
    flag='0',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)

In [38]:
# 实例化trade
trade = Trade(
    # 交易模块需要秘钥
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
    # 实盘：0，虚拟盘：1
    flag='0',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)

In [87]:
# get_ticker = okxSWAP.market.get_ticker('INJ-USDT-SWAP')
# pprint(get_ticker)
# get_books = okxSWAP.market.get_books('INJ-USDT-SWAP', sz=10)
# pprint(get_books)

In [81]:
# config_result = account.get_config()
# config_result

In [ ]:
# 账户模式acctLv
# 1: 简单交易模式
# 2: 单币种保证金模式
# 3: 跨币种保证金模式
# 4: 组合保证金模式
# set_account_level_result = account.set_account_level(acctLv='2')
# set_account_level_result

In [28]:
# 持仓方式: long_short_mode：开平仓模式 net_mode：买卖模式(适用交割/永续)
set_position_mode_result = account.set_position_mode(posMode='long_short_mode')
set_position_mode_result

{'code': '0', 'data': [{'posMode': 'long_short_mode'}], 'msg': ''}

In [30]:
# 永续合约持仓
positions_result = account.get_positions(instType='SWAP')
positions_result

{'code': '0', 'data': [], 'msg': ''}

## 獲取帳戶資金

In [25]:
balance_result = account.get_balance(ccy='USDT')
balance_result['data'][0]['details'][0]['availBal']

'28.6306943513004'

In [70]:
# 永续合约 BTC-USDT-SWAP 全仓
leverage_info_result = account.get_leverage_info(instId='INJ-USDT-SWAP', mgnMode='cross')
leverage_info_result

{'code': '0',
 'data': [{'instId': 'INJ-USDT-SWAP',
   'lever': '1',
   'mgnMode': 'cross',
   'posSide': 'long'},
  {'instId': 'INJ-USDT-SWAP',
   'lever': '1',
   'mgnMode': 'cross',
   'posSide': 'short'}],
 'msg': ''}

## 設定槓桿

In [69]:
set_leverage_result = account.set_leverage(
    instId='INJ-USDT-SWAP', 
    mgnMode='isolated', 
    lever='2.5', 
    posSide='long'
)
set_leverage_result

{'code': '0',
 'data': [{'instId': 'INJ-USDT-SWAP',
   'lever': '2.5',
   'mgnMode': 'isolated',
   'posSide': 'long'}],
 'msg': ''}

## 開倉

In [141]:
# 以2U的价格，逐仓限价开多1张
set_order_swap_limit_open_result = trade.set_order(
    instId = 'INJ-USDT-SWAP',
    ordType = 'limit',
    tdMode = 'isolated',
    posSide = 'long',
    side = 'buy',
    # 用sz的價格買px個幣
    sz = '3', # 已成交 | 委託總量
    px = '1', # 成交均價 | 委託價
)
set_order_swap_limit_open_result

{'code': '0',
 'data': [{'clOrdId': '',
   'ordId': '666635811152478208',
   'sCode': '0',
   'sMsg': 'Order placed',
   'tag': ''}],
 'inTime': '1705210362873291',
 'msg': '',
 'outTime': '1705210362874405'}

In [147]:
order_result = trade.get_order(
    instId = 'INJ-USDT-SWAP',
    ordId = set_order_swap_limit_open_result['data'][0]['ordId']
)
order_result['data'][0]['state']

'canceled'

## 平倉

In [131]:
# 以2U的价格，逐仓限价平多1张个MANA-USDT-SWAP
set_order_swap_limit_close_result = trade.set_order(
    instId='MANA-USDT-SWAP',
    ordType='limit',
    tdMode='isolated',
    posSide='long',
    side='sell',
    sz='1',
    px='1',
)
set_order_swap_limit_close_result

{'code': '1',
 'data': [{'clOrdId': '',
   'ordId': '',
   'sCode': '51169',
   'sMsg': 'Order failed. You don’t have any positions in this contract that can be closed. ',
   'tag': ''}],
 'inTime': '1705210239133849',
 'msg': 'All operations failed',
 'outTime': '1705210239134330'}

## 取消訂單

In [112]:
# 获取币币交易未成单的订单列表
orders_pending_result = trade.get_orders_pending()
print(orders_pending_result['data'][0]['ordId'])

# 按照instId与ordId取消订单
set_cancel_order_result = trade.set_cancel_order(
    instId = 'INJ-USDT-SWAP', ordId = orders_pending_result['data'][0]['ordId']
)
set_cancel_order_result

666628791959568385


{'code': '0',
 'data': [{'clOrdId': '',
   'ordId': '666628791959568385',
   'sCode': '0',
   'sMsg': ''}],
 'inTime': '1705208699254779',
 'msg': '',
 'outTime': '1705208699256329'}

## 市價全平

In [144]:
# MANA-USDT-SWAP 逐仓 多仓 市价全屏
set_close_position_result = trade.set_close_position(
    instId = 'INJ-USDT-SWAP',
    posSide = 'long',
    mgnMode = 'isolated'
)
set_close_position_result

{'code': '51023', 'data': [], 'msg': "Position doesn't exist"}

## 邏輯
1. 獲取當下價格計算和前幾分鐘前的價格差
2. 若價格漲跌超過5%，被視為不合理
3. 使用25%資金開倉
4. 追蹤開倉狀態，若成功則開限價平倉(回調2%)，反之則重新計算價格並修改訂單
5. 持續追蹤價格，若漲跌增加到7.5，持續開倉，槓桿增加0.5
6. 使用25%資金開倉，重複第四點

In [151]:
# market = Market_okx(key = '', secret = '', passphrase = '', flag = '0')
# 获取所有产品行情信息
result = market.get_history_index_candles(instId = "INJ-USDT", bar = "5m", limit = "10")
result['data']

val = (float(result['data'][0][1]) - float(result['data'][9][1]))/float(result['data'][9][1])*100 #最新資料減第十個資料漲跌百分比
print(f"漲幅: {round(val, 3)}%")

balance_result = account.get_balance(ccy='USDT')
bal = balance_result['data'][0]['details'][0]['availBal']
print("USDT balance:", bal)

# 設定每次交易只拿資產的 25% 操作
# 總共可買入的幣，乘以 25%
# 總共買入 px = total 個，價格為 result['data'][0][1]
total = float(bal)/float(result['data'][0][1])*0.25

if val > 5: # 漲

    # 開空
    set_order_swap_limit_open_result = trade.set_order(
        instId = 'INJ-USDT-SWAP',
        ordType = 'limit',
        tdMode = 'isolated',# 持仓方式 isolated：逐仓 cross：全仓
        posSide = 'short',
        side = 'buy',
        sz = result['data'][0][1], # 已成交 | 委託總量
        px = str(total),
    )
    
    # 等待3秒，看有沒有持倉
    time.sleep(5)

    # 追蹤訂單
    order_result = trade.get_order(
        instId = 'INJ-USDT-SWAP',
        ordId = set_order_swap_limit_open_result['data'][0]['ordId']
    )
    # 如果已經持倉
    if order_result['data'][0]['state'] == "filled": # 需更改這裡的狀態結果，持倉並非filled
        # 設置平倉價格為當下的價格上漲2%
        cover = float(result['data'][0][1])*1.02

        set_order_swap_limit_close_result = trade.set_order(
            instId = 'INJ-USDT-SWAP',
            ordType = 'limit',
            tdMode = 'isolated',
            posSide = 'short',
            side = 'sell',
            sz = str(cover),
            px = str(total),
        )
    # 若沒有成功持倉，取消訂單，等待下一個迴圈重新判斷
    else:
        
        if order_result['data'][0]['state'] == "filled":

            # 按照instId与ordId取消订单
            set_cancel_order_result = trade.set_cancel_order(
                instId = 'INJ-USDT-SWAP', ordId = orders_pending_result['data'][0]['ordId']
            )

        
elif val < -5: # 跌

    # 開多
    set_order_swap_limit_open_result = trade.set_order(
        instId = 'INJ-USDT-SWAP',
        ordType = 'limit',
        tdMode = 'isolated',# 持仓方式 isolated：逐仓 cross：全仓
        posSide = 'long',
        side = 'buy',
        sz = result['data'][0][1], # 已成交 | 委託總量
        px = str(total),
    )
    
    # 等待3秒，看有沒有持倉
    time.sleep(5)

    # 追蹤訂單
    order_result = trade.get_order(
        instId = 'INJ-USDT-SWAP',
        ordId = set_order_swap_limit_open_result['data'][0]['ordId']
    )
    # 如果已經持倉
    if order_result['data'][0]['state'] == "filled": # 需更改這裡的狀態結果，持倉並非filled
        # 設置平倉價格為當下的價格下跌2%
        cover = float(result['data'][0][1])*0.08

        # 以2U的价格，逐仓限价平多1张个MANA-USDT-SWAP
        set_order_swap_limit_close_result = trade.set_order(
            instId = 'MANA-USDT-SWAP',
            ordType = 'limit',
            tdMode = 'isolated',
            posSide = 'long',
            side = 'sell',
            sz = str(cover),
            px = str(total),
        )
    # 若沒有成功持倉，取消訂單，等待下一個迴圈重新判斷
    else:
        
        if order_result['data'][0]['state'] == "filled":

            # 按照instId与ordId取消订单
            set_cancel_order_result = trade.set_cancel_order(
                instId = 'INJ-USDT-SWAP', ordId = orders_pending_result['data'][0]['ordId']
            )
            

漲幅: 1.409%
USDT balance: 95.8542709513004
